In [2]:
import yfinance as yf
import pandas as pd

In [3]:
df = pd.read_csv('unfiltered_etfs.csv', header = None, names = ['ETF'])

tickers = df["ETF"].tolist()

start_date = '2015-01-01'
end_date = '2024-12-31'

etf_downloaded = yf.download(tickers = tickers, start = start_date,end = end_date, group_by = 'ticker')

/var/folders/qd/xclxh05s0x9509nblm6z434w0000gp/T/ipykernel_8675/210796664.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  etf_downloaded = yf.download(tickers = tickers, start = start_date,end = end_date, group_by = 'ticker')
[*********************100%***********************]  916 of 916 completed

87 Failed downloads:
['PEVC', 'LCOW', 'LQPE', 'BOBP', 'QQWZ', 'SMDX', 'BRKC', 'FMTM', 'GEND', 'TSEL', 'STOX', 'WZRD', 'GQGU', 'BKCG', 'JDVL', 'DAK', 'CLCV', 'ASLV', 'XOEF', 'AFSC', 'LCAP', 'CGGG', 'XUDV', 'SFTY', 'BLUC', 'BASG', 'BLGR', 'ALIL', 'JXX', 'RSMV', 'AFOS', 'BLUX', 'QIDX', 'EGLE', 'XDIV', 'LST', 'SPYA', 'PVEX', 'LOGO', 'BASV', 'FMKT', 'EBI', 'PRXG', 'FFOX', 'HGRO', 'FLAG', 'CGVV', 'DSPY', 'ASCE', 'MPLY', 'MAVF', 'EPSB', 'EPSV', 'DGLO', 'SEMG', 'LGDX', 'TEXN', 'RUSC', 'AAUS', 'RNIN', 'AGRW', 'SPXM', 'EPMV', 'LSVD', 'BVAL', 'PRXV', 'CLCG', 'CSTK', 'SPXD', 'HWSM', 'TOV', 'JUSA', 'DIVN', 'AVUQ', 'CGMM', 'TOPC', 'EPMB', 'CCFE', 'INVN', 'ALRG', 'ABIG

In [4]:
# Checking the amount of tickers that made it through to the dataset
ticker_list = etf_downloaded.columns.get_level_values(0).unique().tolist()
print(len(ticker_list)) # 916 columns

# ad_close stores close prices
adj_close = etf_downloaded.xs('Close', level=1, axis=1)
print(len(adj_close.columns)) # still 916 columns

# dropping all tickers that have a null first value
# this means that all tickers that don't exist at the start date will be dropped
# and all tickers that failed to download will be dropped
adj_close = adj_close.dropna(axis=1, subset=[adj_close.index[0]])

print(len(adj_close.columns)) # 209 columns remain 


916
916
209


In [5]:
# Double-checking all tickers are recognized by yfinance

tickers = adj_close.columns

def split_supported(tickers):
    ok, bad = [], []
    for t in tickers:
        try:
            # cheap existence check: last day of data if Yahoo knows the symbol
            if not yf.Ticker(t).history(period="1d", auto_adjust=False).empty:
                ok.append(t)
            else:
                bad.append(t)
        except Exception:
            bad.append(t)
    return ok, bad

ok_tickers, bad_tickers = split_supported(tickers)
print(f"Supported on Yahoo: {len(ok_tickers)} | Not found: {len(bad_tickers)}")

# Results:
# 1 ticker not found - CSD, Rest of the 208 are yfinance supported
# Upon program re-run, csd has been found

# Previous results with 87 failed to download: 10 tickers were not recgonized by Yahoo Finance (ABLS,ALRG,CSTK,EGLE,EPMB,EPMV,EPSB,EPSV,LQPE,PEVC), 77 tickers were recognized by Yahoo Finance


Supported on Yahoo: 209 | Not found: 0


In [6]:
# More cleaning and data inspection 

# adj_close = adj_close.drop(columns="CSD")
# tickers = adj_close.columns
# print(len(tickers))

new_etf = etf_downloaded[tickers]

print(adj_close.isna().sum().sum())

for field in ['Open','Close','High','Low','Volume']:
    sub = new_etf.xs(field, level=1, axis=1)
    print(f"\n--- {field} ---")
    print("Shape:", sub.shape)
    print("Total NaNs:", sub.isna().sum().sum())
    print("Tickers with NaNs:", (sub.isna().sum() > 0).sum())

# There are no null values anywhere


0

--- Open ---
Shape: (2515, 209)
Total NaNs: 0
Tickers with NaNs: 0

--- Close ---
Shape: (2515, 209)
Total NaNs: 0
Tickers with NaNs: 0

--- High ---
Shape: (2515, 209)
Total NaNs: 0
Tickers with NaNs: 0

--- Low ---
Shape: (2515, 209)
Total NaNs: 0
Tickers with NaNs: 0

--- Volume ---
Shape: (2515, 209)
Total NaNs: 0
Tickers with NaNs: 0


In [18]:
volume = new_etf.xs("Volume", level = 1, axis = 1)
close = new_etf.xs("Close", level = 1, axis = 1)
avg_volume = volume.mean()
avg_close = close.mean()

min_volume = 500000
min_close = 10

tickers = avg_volume[(avg_volume >= min_volume) & (avg_close >= min_close)].index.tolist()

tickers = list(dict.fromkeys(tickers))
print(len(tickers))
print(avg_close.describe())
print(avg_close.sort_values())

filtered_etfs = pd.DataFrame(tickers)

filtered_etfs.to_csv('filtered_etfs.csv', index = False)

62
count    209.000000
mean      71.888830
std       56.090973
min       12.366706
25%       37.061499
50%       55.425400
75%       79.063124
max      362.952219
dtype: float64
Ticker
SCHB     12.366706
SCHX     12.401268
SCHG     12.723680
FNDB     13.275100
FNDX     13.403668
           ...    
DIA     252.921566
VOO     285.917902
SPY     311.985574
IVV     312.738510
MDY     362.952219
Length: 209, dtype: float64
